In [1]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import sys
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.6.4
sys.version_info(major=3, minor=7, micro=12, releaselevel='final', serial=0)
matplotlib 3.5.2
numpy 1.21.6
pandas 1.3.5
sklearn 1.0.2
tensorflow 2.6.4
keras.api._v2.keras 2.6.0


In [2]:
!cat /kaggle/input/10-monkey-species/monkey_labels.txt

Label,  Latin Name           , Common Name                   , Train Images , Validation Images
n0   , alouatta_palliata	 , mantled_howler                , 131          , 26
n1   , erythrocebus_patas	 , patas_monkey                  , 139          , 28
n2   , cacajao_calvus	     , bald_uakari                   , 137          , 27
n3   , macaca_fuscata	     , japanese_macaque              , 152          , 30
n4   , cebuella_pygmea	     , pygmy_marmoset                , 131          , 26
n5   , cebus_capucinus	     , white_headed_capuchin         , 141          , 28
n6   , mico_argentatus	     , silvery_marmoset              , 132          , 26
n7   , saimiri_sciureus	     , common_squirrel_monkey        , 142          , 28
n8   , aotus_nigriceps	     , black_headed_night_monkey     , 133          , 27
n9   , trachypithecus_johnii , nilgiri_langur                , 132          , 26














In [3]:
train_dir = "/kaggle/input/10-monkey-species/training/training"
valid_dir = "/kaggle/input/10-monkey-species/validation/validation"
label_file = "/kaggle/input/10-monkey-species/monkey_labels.txt"
print(os.path.exists(train_dir))
print(os.path.exists(valid_dir))
print(os.path.exists(label_file))

print(os.listdir(train_dir))
print(os.listdir(valid_dir))

True
True
True
['n8', 'n6', 'n7', 'n1', 'n0', 'n2', 'n5', 'n4', 'n9', 'n3']
['n8', 'n6', 'n7', 'n1', 'n0', 'n2', 'n5', 'n4', 'n9', 'n3']


In [4]:
labels = pd.read_csv(label_file, header=0)
print(labels)

   Label     Latin Name              Common Name                     \
0  n0         alouatta_palliata\t    mantled_howler                   
1  n1        erythrocebus_patas\t    patas_monkey                     
2  n2        cacajao_calvus\t        bald_uakari                      
3  n3        macaca_fuscata\t        japanese_macaque                 
4  n4       cebuella_pygmea\t        pygmy_marmoset                   
5  n5       cebus_capucinus\t        white_headed_capuchin            
6  n6       mico_argentatus\t        silvery_marmoset                 
7  n7      saimiri_sciureus\t        common_squirrel_monkey           
8  n8       aotus_nigriceps\t        black_headed_night_monkey        
9  n9       trachypithecus_johnii    nilgiri_langur                   

    Train Images    Validation Images  
0             131                  26  
1             139                  28  
2             137                  27  
3             152                  30  
4             131 

In [5]:
height = 128
width = 128
channels = 3
batch_size = 64
num_classes = 10
# 图像预处理
train_data_gene = keras.preprocessing.image.ImageDataGenerator(
    rescale=1. / 255,  # 图像中的每个像素点都乘以1/255
    rotation_range=40,  # 图像随机旋转角度范围
    width_shift_range=0.2,  # 水平位移 0-20%之间选择做偏移
    height_shift_range=0.2,  # 垂直方向位移 <1是比例 >=1是像素
    shear_range=0.2,  # 剪切强度 逆时针剪切角，以度为单位
    zoom_range=0.2,  # 缩放范围[lower, upper] 浮点数表示范围[1-zoom_range, 1+zoom_range]
    horizontal_flip=True,  # 水平随机翻转
    fill_mode='nearest'  # 边界外填充 nearest(默认) reflect wrap
)
# 读取图像
train_generator = train_data_gene.flow_from_directory(train_dir,
                                                      # 所有图像调整处理后大小
                                                      target_size=(height, width),
                                                      # 分批 每批图像的数量
                                                      batch_size=batch_size,
                                                      # 用于洗牌和转换的可选随机种子 保证重复运行洗牌结果相同
                                                      seed=7, shuffle=True,
                                                      # 分类模式 categorical生成2D one-hot编码的标签
                                                      # sparse 生成1D整数标签 此时损失计算函数需要改为sparse_categorical_crossentropy
                                                      class_mode='categorical')
# 验证集只需像素点的缩小处理
valid_data_gene = keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255)
valid_generator = valid_data_gene.flow_from_directory(
    valid_dir,
    target_size=(height, width),
    batch_size=batch_size,
    shuffle=False,
    class_mode='categorical')
train_num = train_generator.samples
valid_num = valid_generator.samples
print(train_num, valid_num)

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.
1098 272


In [6]:
for _ in range(2):
    x, y = train_generator.next()
    print(x.shape, y.shape)
    print(y)

(64, 128, 128, 3) (64, 10)
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.

In [7]:
model = keras.models.Sequential([
    # 第一组卷积
    keras.layers.Conv2D(
        filters=32, kernel_size=3, padding='same',
        activation='relu', input_shape=(height, width, channels), ),
    keras.layers.Conv2D(filters=32, kernel_size=3,
                        padding='same', activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    # 第二组翻倍
    keras.layers.Conv2D(
        filters=64, kernel_size=3, padding='same',
        activation='relu', ),
    keras.layers.Conv2D(filters=64, kernel_size=3,
                        padding='same', activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    # 第三组再翻倍
    keras.layers.Conv2D(
        filters=128, kernel_size=3, padding='same',
        activation='relu', ),
    keras.layers.Conv2D(filters=3128, kernel_size=3,
                        padding='same', activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Flatten(),
    # 展平后和全连接层做连接
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(num_classes, activation='softmax')])
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

2022-07-26 13:02:44.263054: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-26 13:02:44.372344: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-26 13:02:44.373327: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-26 13:02:44.375205: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [8]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 128)       7

In [9]:
print(len(model.layers))
print(model.layers)

12
[<keras.layers.convolutional.Conv2D object at 0x7fac9d5e7bd0>, <keras.layers.convolutional.Conv2D object at 0x7fac9d67c4d0>, <keras.layers.pooling.MaxPooling2D object at 0x7fac9d67cc10>, <keras.layers.convolutional.Conv2D object at 0x7fac9e54ca90>, <keras.layers.convolutional.Conv2D object at 0x7fac9e54cbd0>, <keras.layers.pooling.MaxPooling2D object at 0x7fac9d5d6410>, <keras.layers.convolutional.Conv2D object at 0x7fac9d5d6910>, <keras.layers.convolutional.Conv2D object at 0x7fac9d5d6d10>, <keras.layers.pooling.MaxPooling2D object at 0x7fac9d6e2390>, <keras.layers.core.Flatten object at 0x7fac9d6e28d0>, <keras.layers.core.Dense object at 0x7fac9d6e29d0>, <keras.layers.core.Dense object at 0x7fac9d6e2d50>]


In [ ]:
epochs = 100
history = model.fit_generator(train_generator,
                              # 显式指定
                              steps_per_epoch=train_num // batch_size,
                              epochs=epochs, validation_data=valid_generator,
                              validation_steps=valid_num // batch_size)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-07-26 13:02:48.803932: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2022-07-26 13:02:53.530449: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


17/17 [==============================] - 55s 3s/step - loss: 2.7636 - accuracy: 0.1083 - val_loss: 2.2875 - val_accuracy: 0.1172
Epoch 2/100
17/17 [==============================] - 35s 2s/step - loss: 2.2792 - accuracy: 0.1451 - val_loss: 2.2281 - val_accuracy: 0.1875
Epoch 3/100
17/17 [==============================] - 36s 2s/step - loss: 2.1998 - accuracy: 0.1896 - val_loss: 2.0711 - val_accuracy: 0.2969
Epoch 4/100
17/17 [==============================] - 36s 2s/step - loss: 2.0716 - accuracy: 0.2273 - val_loss: 1.9434 - val_accuracy: 0.2812
Epoch 5/100
17/17 [==============================] - 36s 2s/step - loss: 2.0448 - accuracy: 0.2350 - val_loss: 1.9654 - val_accuracy: 0.2852
Epoch 6/100
17/17 [==============================] - 36s 2s/step - loss: 1.9777 - accuracy: 0.2631 - val_loss: 1.8749 - val_accuracy: 0.3164
Epoch 7/100
17/17 [==============================] - 36s 2s/step - loss: 1.9805 - accuracy: 0.2679 - val_loss: 1.8548 - val_accuracy: 0.3398
Epoch 8/100
17/17 [======

In [ ]:
print(history.history)

In [ ]:
def plot_learning_curves(history, label, epochs, min_value, max_value):
    data = {}
    data[label] = history.history[label]
    data['val' + data] = history.history['val' + label]
    pd.DataFrame(data).plot(figsize=(8, 5))
    plt.grid(True)
    plt.axis([0, epochs, min_value, max_value])
    plt.show()


plot_learning_curves(history, 'accuracy', epochs, 0, 1)
plot_learning_curves(history, 'loss', epochs, 1.5, 2.5)
